In [1]:
# data manipulation library
import pandas as pd
import numpy as np
#data visualization library
import matplotlib.pyplot as plt
import seaborn as sns
# ignore warning masseges library
import warnings 
warnings.filterwarnings('ignore')
#import logging
import logging
logging.basicConfig(level=logging.INFO,
                   format ='%(asctime)s - %(levelname)s - %(message)s',
                   filemode = 'w',
                   filename = 'model.log',force = True)

In [2]:
logging.info('IMPORT DATASET FOR MODEL BUILDING')

In [3]:
url = 'https://raw.githubusercontent.com/varunr1909/CementPrediction_Model/refs/heads/main/Concrete_Data.csv'

df = pd.read_csv(url)
df.sample(frac=1)

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
489,387.0,20.0,94.0,157.0,13.9,938.0,845.0,3,25.51
817,525.0,0.0,0.0,189.0,0.0,1125.0,613.0,14,48.40
247,238.1,0.0,94.1,186.7,7.0,949.9,847.0,56,39.59
103,425.0,106.3,0.0,153.5,16.5,852.1,887.1,7,49.20
589,339.2,0.0,0.0,185.7,0.0,1069.2,754.3,28,31.90
...,...,...,...,...,...,...,...,...,...
751,540.0,0.0,0.0,173.0,0.0,1125.0,613.0,7,52.61
164,425.0,106.3,0.0,153.5,16.5,852.1,887.1,91,65.20
118,425.0,106.3,0.0,153.5,16.5,852.1,887.1,28,60.29
61,266.0,114.0,0.0,228.0,0.0,932.0,670.0,270,51.73


In [26]:
from collections import OrderedDict
stats=[]
for i in df.columns:
    numerical_stats = OrderedDict({
        'feature':i ,
        'min':df[i].min(),
        'max':df[i].max(),
        'mean':df[i].mean(),
        'median':df[i].quantile(0.50),
        'quantile(0.25)':df[i].quantile(0.25),
        'quantile(0.75)':df[i].quantile(0.75),
        'Standerd deviation':df[i].std(),                               
        'IQR':df[i].quantile(0.75)-df[i].quantile(0.25),
        'skewness':df[i].skew(),
        'kurtosis':df[i].kurtosis()
        })
    stats.append(numerical_stats)
    report = pd.DataFrame(stats)

report
#if your data having positive skewed or negative skeweed in that scenario never ever impute with mean

,feature,min,max,mean,median,quantile(0.25),quantile(0.75),Standerd deviation,IQR,skewness,kurtosis
0,Cement (component 1)(kg in a m^3 mixture),102.00,540.0,281.167864,272.900,192.375,350.000,104.506364,157.625,0.509481,-0.520652
1,Blast Furnace Slag (component 2)(kg in a m^3 m...,0.00,359.4,73.895825,22.000,0.000,142.950,86.279342,142.950,0.800717,-0.508175
2,Fly Ash (component 3)(kg in a m^3 mixture),0.00,200.1,54.188350,0.000,0.000,118.300,63.997004,118.300,0.537354,-1.328746
3,Water (component 4)(kg in a m^3 mixture),121.80,247.0,181.567282,185.000,164.900,192.000,21.354219,27.100,0.074628,0.122082
4,Superplasticizer (component 5)(kg in a m^3 mix...,0.00,32.2,6.204660,6.400,0.000,10.200,5.973841,10.200,0.907203,1.411269
5,Coarse Aggregate (component 6)(kg in a m^3 mi...,801.00,1145.0,972.918932,968.000,932.000,1029.400,77.753954,97.400,-0.040220,-0.599016
6,Fine Aggregate (component 7)(kg in a m^3 mixture),594.00,992.6,773.580485,779.500,730.950,824.000,80.175980,93.050,-0.253010,-0.102177
7,Age (day),1.00,365.0,45.662136,28.000,7.000,56.000,63.169912,49.000,3.269177,12.168989
8,"Concrete compressive strength(MPa, megapascals)",2.33,82.6,35.817961,34.445,23.710,46.135,16.705742,22.425,0.416977,-0.313725


In [5]:
#from given information mean is not eqal to median then data is non normal distrubuted
df.describe()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Cement (component 1)(kg in a m^3 mixture)              1030 non-null   float64
 1   Blast Furnace Slag (component 2)(kg in a m^3 mixture)  1030 non-null   float64
 2   Fly Ash (component 3)(kg in a m^3 mixture)             1030 non-null   float64
 3   Water  (component 4)(kg in a m^3 mixture)              1030 non-null   float64
 4   Superplasticizer (component 5)(kg in a m^3 mixture)    1030 non-null   float64
 5   Coarse Aggregate  (component 6)(kg in a m^3 mixture)   1030 non-null   float64
 6   Fine Aggregate (component 7)(kg in a m^3 mixture)      1030 non-null   float64
 7   Age (day)                                              1030 non-null   int64  
 8   Concrete compressive strength(MPa, megapascals)  

In [14]:
df.columns

Index(['Cement (component 1)(kg in a m^3 mixture)',
       'Blast Furnace Slag (component 2)(kg in a m^3 mixture)',
       'Fly Ash (component 3)(kg in a m^3 mixture)',
       'Water  (component 4)(kg in a m^3 mixture)',
       'Superplasticizer (component 5)(kg in a m^3 mixture)',
       'Coarse Aggregate  (component 6)(kg in a m^3 mixture)',
       'Fine Aggregate (component 7)(kg in a m^3 mixture)', 'Age (day)',
       'Concrete compressive strength(MPa, megapascals) '],
      dtype='object')

In [18]:
import statsmodels.api as sm
X = df.drop(columns='Concrete compressive strength(MPa, megapascals) ',axis = 1)
y = df['Concrete compressive strength(MPa, megapascals) ']

In [22]:
X = sm.add_constant(X)

In [24]:
model = sm.OLS(y,X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                           OLS Regression Results                                           
============================================================================================================
Dep. Variable:     Concrete compressive strength(MPa, megapascals)    R-squared:                       0.616
Model:                                                          OLS   Adj. R-squared:                  0.613
Method:                                               Least Squares   F-statistic:                     204.3
Date:                                              Wed, 28 May 2025   Prob (F-statistic):          6.29e-206
Time:                                                      19:45:10   Log-Likelihood:                -3869.0
No. Observations:                                              1030   AIC:                             7756.
Df Residuals:                                                  1021   BIC:                             7800.
Df Model:                                                         8                                         
Covariance Type:                                          nonrobust                                         
=========================================================================================================================
                                                            coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------------
const                                                   -23.3312     26.586     -0.878      0.380     -75.500      28.837
Cement (component 1)(kg in a m^3 mixture)                 0.1198      0.008     14.113      0.000       0.103       0.136
Blast Furnace Slag (component 2)(kg in a m^3 mixture)     0.1039      0.010     10.247      0.000       0.084       0.124
Fly Ash (component 3)(kg in a m^3 mixture)                0.0879      0.013      6.988      0.000       0.063       0.113
Water  (component 4)(kg in a m^3 mixture)                -0.1499      0.040     -3.731      0.000      -0.229      -0.071
Superplasticizer (component 5)(kg in a m^3 mixture)       0.2922      0.093      3.128      0.002       0.109       0.476
Coarse Aggregate  (component 6)(kg in a m^3 mixture)      0.0181      0.009      1.926      0.054      -0.000       0.037
Fine Aggregate (component 7)(kg in a m^3 mixture)         0.0202      0.011      1.887      0.059      -0.001       0.041
Age (day)                                                 0.1142      0.005     21.046      0.000       0.104       0.125
==============================================================================
Omnibus:                        5.378   Durbin-Watson:                   1.282
Prob(Omnibus):                  0.068   Jarque-Bera (JB):                5.304
Skew:                          -0.174   Prob(JB):                       0.0705
Kurtosis:                       3.045   Cond. No.                     1.06e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.06e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""